In [ ]:
# Paso 1: Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
# Paso 2: Preparar las rutas de entrada y salida
# Ruta del video y la salida en Google Drive
# input_path = '/content/drive/MyDrive/Colab Notebooks/CursoIA/Shopping Mall720p-h264.mp4'
input_path = '/content/drive/MyDrive/Colab Notebooks/CursoIA/shoppingmall.mp4'
output_path = '/content/drive/My Drive/Colab Notebooks/CursoIA/output.mp4'

path = '/content/drive/My Drive/Colab Notebooks/CursoIA/'

In [ ]:
# Comprobamos que los archivos existen
import os
if not os.path.exists(input_path):
    raise FileNotFoundError(f"El archivo de video {input_path} no fue encontrado.")



In [ ]:
# Paso 3: Instalación del módulo Ultralytics:
# El módulo Ultralytics es una biblioteca de inteligencia artificial
# enfocada en la visión por computadora, que se especializa en la
# implementación de modelos de detección de objetos y segmentación de imágenes

%pip install ultralytics

# Instalación modulo cvzone
# El paquete cvzone es una biblioteca de visión por computadora
# que facilita la implementación de funciones de procesamiento
# de imágenes y técnicas de inteligencia artificial

%pip install cvzone

!pip install deep_sort_realtime # Install the deep_sort_realtime module


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26298 sha256=3e87004f28dc33b49d23486945462464fcf7f948a40b5019871ed83f36fb12e1
  Stored in directory: /root/.cache/pip/wheels/2c/9f/b3/92e945ac4a71bf727a92463f38155cc5a4fa49c5010b38ec4c
Successfully built cvzone
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 58.3 MB/s eta 0:00:00


In [ ]:
# Paso 4: Inicializar modelo y cargar datos

import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
import sys
sys.path.append(path)
from deep_sort_realtime.deepsort_tracker import DeepSort

import cvzone

# Inicializar el modelo YOLO
print("Cargando modelo YOLO...")
model = YOLO('yolov8n.pt')



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Cargando modelo YOLO...


100%|██████████| 6.25M/6.25M [00:00<00:00, 115MB/s]


In [ ]:
# Paso 5: Definición de funcione
# Función para verificar el cruce de la persona dentro de un área poligonal

import numpy as np
import matplotlib.path as mpath

def esta_dentro_area(centroide_x, centroide_y, area_coords, entra_sale):

    # Definir el centroide de la persona
    centroide_persona = (centroide_x, centroide_y)
    # Crear el objeto Path
    path = mpath.Path(area_coords)
    dentro_del_area = path.contains_point(centroide_persona)
    if dentro_del_area:
      print(f"Coordenadas centroides {entra_sale}:\n"
      f"  centroide_x : {centroide_x} "
      f"  centroide_y : {centroide_y} \n"
      f"  centroide_persona : {centroide_persona} \n"
      f" Esta? {dentro_del_area}\n"
      f"  area_polygon : {area_coords}\n")

    return dentro_del_area
    # return dentro_del_area >= 0  # True si está dentro o en el borde del área



In [ ]:
# Paso 6: Funcion para identificar que tan solapada esta una area frente a otra

from itertools import combinations  # Importa combinaciones, aunque no se utiliza en este código.

# Función de supresión no máxima (NMS)
def non_max_suppression(detections, iou_threshold=0.5):
    """
    Aplica supresión no máxima para eliminar detecciones redundantes basadas en el IoU (Intersection over Union).

    Args:
        detections: Lista de detecciones en formato [(x, y, w, h), score].
        iou_threshold: Umbral para decidir si dos cuadros se consideran redundantes.

    Returns:
        Lista de detecciones finales después de aplicar NMS.
    """
    if len(detections) == 0:
        return []  # Si no hay detecciones, retorna una lista vacía.

    # Ordena las detecciones por la confianza (score) de forma descendente.
    detections = sorted(detections, key=lambda x: x[1], reverse=True)

    final_detections = []  # Lista donde se almacenarán las detecciones filtradas.

    # Mientras existan detecciones por procesar
    while detections:
        # Toma la detección con mayor confianza (la primera en la lista).
        current = detections.pop(0)
        final_detections.append(current)  # Agrega esta detección a la lista final.

        # Filtra las detecciones restantes para eliminar las que tienen IoU alto con `current`.
        detections = [
            d for d in detections
            if iou(current[0], d[0]) < iou_threshold  # Conserva solo las detecciones con IoU menor al umbral.
        ]

    return final_detections  # Retorna la lista de detecciones finales.

# Función para calcular el IoU (Intersection over Union)

def iou(box1, box2):
    """
    Calcula el coeficiente de IoU entre dos cuadros delimitadores (bounding boxes).

    Args:
        box1: Coordenadas del primer cuadro en formato (x, y, w, h).
        box2: Coordenadas del segundo cuadro en formato (x, y, w, h).

    Returns:
        Valor de IoU (float) entre 0 y 1.
    """
    # Desempaqueta las coordenadas y dimensiones de los cuadros.
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2

    # Calcula las coordenadas de la intersección entre los dos cuadros.
    xi1 = max(x1, x2)  # Coordenada x del borde izquierdo de la intersección.
    yi1 = max(y1, y2)  # Coordenada y del borde superior de la intersección.
    xi2 = min(x1 + w1, x2 + w2)  # Coordenada x del borde derecho de la intersección.
    yi2 = min(y1 + h1, y2 + h2)  # Coordenada y del borde inferior de la intersección.

    # Calcula el área de la intersección.
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)  # Asegura que no haya áreas negativas.

    # Calcula las áreas de los cuadros individuales.
    box1_area = w1 * h1  # Área del primer cuadro.
    box2_area = w2 * h2  # Área del segundo cuadro.

    # Calcula el IoU como el área de la intersección dividida entre el área de la unión.
    return inter_area / (box1_area + box2_area - inter_area)


In [ ]:

nuevo_ancho, nuevo_alto = 1020, 500  # Dimensiones deseadas para cada cuadro

# Configuración del video
cap = cv2.VideoCapture(input_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 20.0, (nuevo_ancho, nuevo_alto))  # Tamaño corregido

# Configuración de Deep SORT
# max_cosine_distance:
# Este parámetro controla la tolerancia en la similitud de características (apariencia) entre
# cuadros consecutivos. Un valor mayor permite al tracker aceptar detecciones que difieren más.
# max_age:
# Este parámetro controla cuántos cuadros puede persistir un track sin una nueva detección antes
# de ser eliminado. Si es muy bajo, un track puede desaparecer cuando la persona está agachada y
# reaparecer como un nuevo track.
# n_init:
# Controla cuántos cuadros consecutivos debe ser detectado un objeto para ser confirmado como un
# track. Incrementar este valor reducirá el riesgo de que pequeñas interrupciones generen nuevos IDs.
tracker = DeepSort(max_age=20, n_init=5, max_cosine_distance=0.2)
# tracker = DeepSort(max_age=30, n_init=8, max_cosine_distance=0.4)

# Inicializar seguimiento de objetos y áreas
area_entrada = np.array([[640, 182], [640, 199], [959, 317], [959, 300]])  # Entrada
area_salida = np.array([[631, 222], [631, 235], [927, 353], [937, 342]])  # Salida

# Convertir area_entrada y area_salida al formato esperado por cv2.polylines para poder mostrar en el video resultante
area_entrada_polylines = area_entrada.reshape((-1, 1, 2))
area_salida_polylines = area_salida.reshape((-1, 1, 2))

# Contadores de entrada y salida
entradas = 0
salidas = 0

# Diccionario para rastrear el estado anterior de cada objeto
estado_anterior = {}
estado_objeto = {}
objetos_entrando = {}
objetos_saliendo = {}
contador = 0

while cap.isOpened():
  contador += 1
  ret, frame = cap.read()
  # if contador <= 250:
  #   continue

  if not ret:# or contador > 650:
    break

  # Redimensionar el cuadro a 1020 x 500
  frame = cv2.resize(frame, (nuevo_ancho, nuevo_alto))

  # Realizar detección con YOLOv8
  results = model(frame)
  detections = []

  confidence_threshold = 0.5  # Umbral de confianza

  for result in results:
    for box in result.boxes:
      if box.cls == 0 and box.conf[0] > confidence_threshold:  # Solo para la clase de personas
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Coordenadas del bounding box
        confidence = box.conf[0]  # Confianza de la detección. Un valor más alto significa que el modelo está más seguro de que la detección es correcta.

        # Ajustar coordenadas si están fuera del rango
        x1 = max(0, min(x1, 1019))  # Limitar x1 entre 0 y 1019
        y1 = max(0, min(y1, 499))   # Limitar y1 entre 0 y 499
        x2 = max(0, min(x2, 1019))  # Limitar x2 entre 0 y 1019
        y2 = max(0, min(y2, 499))   # Limitar y2 entre 0 y 499
        w, h = x2 - x1, y2 - y1
        # Añadir detección al formato esperado por Deep SORT
        detections.append([[x1, y1, w, h], confidence.item(), 0])

  # Filtrar detecciones solapadas
  detections = non_max_suppression(detections)

  # Actualizar el tracker con las detecciones actuales
  tracks = tracker.update_tracks(detections, frame=frame)

  # Dibujar áreas de entrada y salida
  cv2.polylines(frame, [area_entrada_polylines], True, (0, 0, 255), 2)
  cv2.polylines(frame, [area_salida_polylines], True, (0, 255, 120), 2)

  # Procesar cada objeto rastreado
  for track in tracks:

      if not track.is_confirmed():  # Saltar si el track no está confirmado
          continue

      track_id = track.track_id
      x1, y1, x2, y2 = map(int, track.to_ltrb())  # Convertir a coordenadas (x1, y1, x2, y2)

      #centroide_x, centroide_y = (x1 + x2) // 2, (y1 + y2) // 2
      centroide_x, centroide_y = (x2), (y2) #Esquina inferior derecha

      # Verificar si el centroide está dentro del área de entrada o salida
      en_entrada = esta_dentro_area(centroide_x, centroide_y, area_entrada, True)
      en_salida = esta_dentro_area(centroide_x, centroide_y, area_salida, False)

      # Inicializar el estado del objeto si no existe
      if track_id not in estado_anterior:
          estado_anterior[track_id] = {"en_entrada": False, "en_salida": False}

      # Obtener el estado anterior del objeto
      estado_objeto = estado_anterior[track_id]

      # Detectar si hay cruce desde área de salida a entrada (contabilizar como entrada)
      if estado_objeto["en_salida"] and en_entrada:
        if track_id not in objetos_saliendo:  # Verificar si el ID ya está en el diccionario
          objetos_saliendo[track_id] = True  # Agregar el ID al diccionario si no está
          entradas += 1
          print(f"Objeto {track_id} ha entrado.")
          # Reinicia estados anterior por conteo del objeto
          estado_anterior[track_id]["en_salida"] = False
          estado_anterior[track_id]["en_entrada"] = False
        # Detectar si hay cruce desde área de entrada a Salida (contabilizar como salida)
      elif estado_objeto["en_entrada"] and en_salida:
        if track_id not in objetos_entrando:  # Verificar si el ID ya está en el diccionario
          print(f"en entrada {en_entrada} --  en salida {en_salida}  --  objeto ID {track_id} -- estado anterior {estado_anterior} -- estado objeto {estado_objeto}\n")
          objetos_entrando[track_id] = True  # Agregar el ID al diccionario si no está
          salidas += 1
          print(f"Objeto {track_id} ha salido.")
          # Reinicia estados anterior por conteo del objeto
          estado_anterior[track_id]["en_salida"] = False
          estado_anterior[track_id]["en_entrada"] = False

      # Actualizar el estado actual del objeto
      if not estado_anterior[track_id]["en_entrada"]:
        estado_anterior[track_id]["en_entrada"] = en_entrada

      if not estado_anterior[track_id]["en_salida"]:
        estado_anterior[track_id]["en_salida"] = en_salida

      cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
      cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

  cv2.putText(frame, f'Entradas: {entradas} Salidas: {salidas}',
            (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
  # Guardar el cuadro procesado
  out.write(frame)

# Liberar recursos
cap.release()
out.release()
cv2.destroyAllWindows()
print("Procesamiento completado. Video guardado en", output_path)


Se truncaron las últimas líneas 5000 del resultado de transmisión.

0: 320x640 (no detections), 9.4ms
Speed: 2.8ms preprocess, 9.4ms inference, 0.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 10.3ms
Speed: 2.4ms preprocess, 10.3ms inference, 0.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 8.8ms
Speed: 2.6ms preprocess, 8.8ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 11.1ms
Speed: 3.0ms preprocess, 11.1ms inference, 0.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 16.2ms
Speed: 2.5ms preprocess, 16.2ms inference, 0.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 12.2ms
Speed: 2.5ms preprocess, 12.2ms inference, 0.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 9.9ms
Speed: 2.4ms preprocess, 9.9ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x

In [ ]:
# Paso 7: Visualizar video procesado

from IPython.display import HTML
from base64 import b64encode

def display_video(output_path):
    mp4 = open(output_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f'<video width=700 controls><source src="{data_url}" type="video/mp4"></video>')

display_video(output_path)


Output hidden; open in https://colab.research.google.com to view.